# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

import pickle
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import re
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
df.head(2)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
df.select_dtypes(include=['object']).columns

Index(['message', 'original', 'genre'], dtype='object')

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql("SELECT * FROM DisasterResponse", engine)
    X = df['message']
    y = df.drop(['message', 'genre', 'id', 'original'], axis=1)
    
    return X, y
    
def tokenize(message):
    # get list of all urls using regex
    detected_urls = re.findall(url_regex, message)
    
    # replace each url in message string with placeholder
    for url in detected_urls:
        message = message.replace(url, "urlplaceholder")
        
    message=re.sub(r"[^a-zA-Z0-9]"," ",message.lower())
  
    # tokenize text
    tokens = word_tokenize(message)
    
    #removing stopwords from text
    tokens = [tok for tok in tokens if tok not in stopwords.words("english")]

    # lemmatize, normalize case, and remove leading/trailing white space
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


In [6]:
X, y = load_data()
'''
viewing clean data messages 
'''
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti'] 

Is the Hurricane over or is it not over
['hurricane'] 

Looking for someone but no name
['looking', 'someone', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately'] 

says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
model = Pipeline([ ('vect',CountVectorizer(tokenizer=tokenize)),
                      ('tfidf',TfidfTransformer()),
                      ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=15,n_jobs=-1)))
                        ]) 

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
model.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred=model.predict(X_test)
y_pred

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [11]:
for col in range(36):
    print(y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred[:,col]))
    print('-----------------------------------------------------')

related
             precision    recall  f1-score   support

          0       0.68      0.43      0.53      1266
          1       0.84      0.93      0.88      3938
          2       0.35      0.45      0.39        40

avg / total       0.79      0.81      0.79      5244

-----------------------------------------------------
request
             precision    recall  f1-score   support

          0       0.90      0.97      0.94      4349
          1       0.78      0.49      0.60       895

avg / total       0.88      0.89      0.88      5244

-----------------------------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

-----------------------------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.79      0.82      0.81      3113

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
overall_accuracy = (y_pred == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))


Average overall accuracy 94.69% 



### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
pipeline1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f104038b2f0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=0.01, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_ran

In [22]:
parameters = { 
    'vect__max_features' : (None,100),
    'clf__estimator__n_estimators':[50,100],
    'clf__estimator__learning_rate': [0.01,0.001,1.0]
}

cv = GridSearchCV(estimator=pipeline1,param_grid=parameters,verbose=2,n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
cv.fit(X_train,y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=None 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=None, total= 2.3min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=None 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=None, total= 2.3min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=None 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=None, total= 2.2min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=100 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=100, total= 1.6min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=100 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=100, total= 1.6min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=100 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, vect__max_features=100, total= 1.6min
[CV] clf__estimator__learning_rate=0.01,

[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 116.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ator=None,
          learning_rate=0.01, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__max_features': (None, 100), 'clf__estimator__n_estimators': [50, 100], 'clf__estimator__learning_rate': [0.01, 0.001, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [24]:
y_pred_cv = cv.predict(X_test)

In [26]:
y_pred_cv

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [25]:
cv.best_params_

{'clf__estimator__learning_rate': 1.0,
 'clf__estimator__n_estimators': 100,
 'vect__max_features': 100}

In [27]:
for col in range(36):
    print(y_test.columns[col])
    print(classification_report(y_test.iloc[:,col], y_pred_cv[:,col]))
    print('-----------------------------------------------------')

related
             precision    recall  f1-score   support

          0       0.58      0.26      0.36      1873
          1       0.80      0.94      0.86      5934
          2       0.00      0.00      0.00        58

avg / total       0.74      0.77      0.74      7865

-----------------------------------------------------
request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6533
          1       0.77      0.48      0.59      1332

avg / total       0.88      0.89      0.88      7865

-----------------------------------------------------
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      7865

-----------------------------------------------------
aid_related
             precision    recall  f1-score   support

          0       0.74      0.83      0.79      4646

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
'''
for improving a model use
AdaBoost Classifer
'''

In [30]:
pipeline_adaboost = Pipeline([ ('vect',CountVectorizer(tokenizer=tokenize,max_features=100)),
                      ('tfidf',TfidfTransformer()),
                      ('clf',MultiOutputClassifier(AdaBoostClassifier(n_estimators=100,learning_rate=1.0)))
                        ]) 

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [32]:
pipeline_adaboost.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=100, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...ator=None,
          learning_rate=1.0, n_estimators=100, random_state=None),
           n_jobs=1))])

In [33]:
y_pred_adaB=pipeline_adaboost.predict(X_test)
y_pred_adaB

array([[1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [34]:
for col in range(36):
            print(y_test.columns[col])
        
            print(classification_report(y_test.iloc[:,col],y_pred_adaB[:,col] ))
            print("_________________________________________________________________")

related
             precision    recall  f1-score   support

          0       0.58      0.26      0.36      1873
          1       0.80      0.94      0.86      5934
          2       0.00      0.00      0.00        58

avg / total       0.74      0.77      0.74      7865

_________________________________________________________________
request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6533
          1       0.77      0.48      0.59      1332

avg / total       0.88      0.89      0.88      7865

_________________________________________________________________
offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      0.99      0.99      7865

_________________________________________________________________
aid_related
             precision    recall  f1-score   support

          0     

In [36]:
overall_accuracy = (y_pred_adaB == y_test).mean().mean()
print('Average overall accuracy {0:.2f}% \n'.format(overall_accuracy*100))

Average overall accuracy 94.26% 



### 9. Export your model as a pickle file

In [18]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.